# Introduction to Feature Engineering
**Learning Objectives**
  * Improve the accuracy of a model by using feature engineering
  * Understand there's two places to do feature engineering in Tensorflow
    1. Using the `tf.feature_column` module
    2. In the input functions
    
Up until now we've been focusing on Tensorflow mechanics to make sure our code works, we have neglected model performance, which at this point is **9.26 RMSE**. 

In this notebook we'll attempt to improve on that using feature engineering.

In [ ]:
!pip freeze | grep tensorflow==1.12.0 || pip install tensorflow==1.12.0

In [ ]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

## 1) Load Raw Data

These are the same files created in the `create_datasets.ipynb` notebook

In [ ]:
!gsutil cp gs://cloud-training-demos/taxifare/small/*.csv .
!ls -l *.csv

## 2) Train and Evaluate Input Functions

These are the same as before with one additional line of code: a call to `add_engineered_features()` from within the `_parse_row()` function.

In [ ]:
CSV_COLUMN_NAMES = ['fare_amount','dayofweek','hourofday','pickuplon','pickuplat','dropofflon','dropofflat']
CSV_DEFAULTS = [[0.0],[1],[0],[-74.0], [40.0], [-74.0], [40.7]]

def read_dataset(csv_path):
    def _parse_row(row):
        # Decode the CSV row into list of TF tensors
        fields = tf.decode_csv(row, record_defaults=CSV_DEFAULTS)

        # Pack the result into a dictionary
        features = dict(zip(CSV_COLUMN_NAMES, fields))
        
        # NEW: Add engineered features
        features = add_engineered_features(features)
        
        # Separate the label from the features
        label = features.pop('fare_amount') # remove label from features and store

        return features, label
    
    # Create a dataset containing the text lines.
    dataset = tf.data.Dataset.list_files(csv_path) # (i.e. data_file_*.csv)
    dataset = dataset.flat_map(lambda filename:tf.data.TextLineDataset(filename).skip(1))

    # Parse each CSV row into correct (features,label) format for Estimator API
    dataset = dataset.map(_parse_row)
    
    return dataset

def train_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)
      
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
   
    return dataset

def eval_input_fn(csv_path, batch_size=128):
    #1. Convert CSV into tf.data.Dataset  with (features,label) format
    dataset = read_dataset(csv_path)

    #2.Batch the examples.
    dataset = dataset.batch(batch_size)
   
    return dataset

## 3) Feature Engineering: Feature Columns

There are two places in Tensorflow where we can do feature engineering. The first is using the `tf.feature_column` package. This allows us easily 

- bucketize continuous features
- one hot encode categorical features
- create feature crosses

For details on the possible `tf.feature_column` transformations and when to use each see the [official guide](https://www.tensorflow.org/guide/feature_columns).

Let's use `tf.feature_column` to create a feature that shows the combination of day of week and hour of day. This will allow our model to easily learn the difference between say Wednesday at 5pm (rush hour, expect higher fares) and Sunday at 5pm (light traffic, expect lower fares).

Let's also use it to bucketize our latitudes and longitudes because treating them as continuous numbers is misleading to the model.

### Exercise 1

Complete the code below by adding the necessary `tf.feature_column`. Have a look at the various feature columns [on this page](https://www.tensorflow.org/api_docs/python/tf/feature_column) and follow the links to the relevant documentation.

In [ ]:
# 1. One hot encode dayofweek and hourofday
fc_dayofweek = # TODO: Create a categorical feature column for 'dayofweek'
fc_hourofday = # TODO: Create a categorical feature column for 'hourofday'

# 2. Cross features to get combination of day and hour
fc_day_hr = # TODO: Create a new feature column by crossing the categorical features 'fc_dayofweek' and 'fc_hourofday' you created above

# 3. Bucketize latitudes and longitudes
NBUCKETS = 16
latbuckets = np.linspace(38.0, 42.0, NBUCKETS).tolist()
lonbuckets = np.linspace(-76.0, -72.0, NBUCKETS).tolist()
fc_bucketized_plat = # TODO: Create a bucketized feature column for the pickup latitude. Be careful to note that the 'source_column' for a bucketized feature column should a feature column itself. 
fc_bucketized_plon = # TODO: Create a bucketized feature column for the pickup longitude. Be careful to note that the 'source_column' for a bucketized feature column should a feature column itself. 
fc_bucketized_dlat = # TODO: Create a bucketized feature column for the dropoff latitude. Be careful to note that the 'source_column' for a bucketized feature column should a feature column itself. 
fc_bucketized_dlon = # TODO: Create a bucketized feature column for the dropoff longitude. Be careful to note that the 'source_column' for a bucketized feature column should a feature column itself. 

## 4) Feature Engineering: Input Functions

While feature columns are very powerful, what happens when we want to something that there isn't a feature column for?

Recall the input functions recieve csv data, format it, then pass it batch by batch to the model. We can also use input functions to inject arbitrary tensorflow code to manipulate the data.

However, we need to be careful that any transformations we do in one input function, we do for all, otherwise we'll have [training-serving skew](https://developers.google.com/machine-learning/guides/rules-of-ml/#training-serving_skew).

To guard against this we encapsulate all input function feature engineering in a single function, `add_engineered_features()`, and call this function from every input function.

Let's calculate the euclidean distance between the pickup and dropoff points and feed that as a new feature to our model. 

Also it may be useful to know which cardinal direction that distance is in. I suspect that distance is cheaper to travel North/South because in Manhatten streets that run North/South have less stops than streets that run East/West.

### Exercise 2

Create some new engineered features using the `add_engineered_features` function below. This function takes a *dictionary* of features and returns the dictionary with some additional features added.
We want to engineer a new feature which captures the Euclidean distance (the straight-line distance) between the pickup and dropoff. Complete the code below to compute the 
difference in the latitude and the difference in the longitude. Then, use these to compute the Euclidean distance and add that to the features dictionary as well.

In [ ]:
def add_engineered_features(features):
    features['latdiff'] = # TODO: Compute the difference in the pickup latitude and the dropoff latitude
    features['londiff'] = # TODO: Compute the difference in the pickup longitude and the dropoff longitude
    features['euclidean_dist'] = # TODO: Use 'latdiff' and 'londiff' to compute the Euclidean distance between the pickup and dropoff

    return features

## 5) Gather list of Feature Columns

Ultimately our estimator expects a list of feature columns, so let's gather all our engineered features into a single list.

We cannot pass categorical or crossed feature columns directly into a DNN, Tensorflow will give us an error. We must first wrap them using either `indicator_column()` or `embedding_column()`. The former will pass through the one-hot encoded representation as is, the latter will embed the feature into a dense representation of specified dimensionality (the 4th root of the number of categories is a good starting point for number of dimensions). Read more about indicator and embedding columns [here](https://www.tensorflow.org/guide/feature_columns#indicator_and_embedding_columns).


### Exercise 3

Create a list containing all the necessary feature columns for our model, including the new feature columns we created above. 

Hint: You will need to use an `indicator_column()` to wrap any categorial or crossed feature columns.

In [ ]:
feature_cols = [
  #1. Engineered using tf.feature_column module
  # TODO: Add here all the features columns we created above using the tf.feature_column module
  #2. Engineered in input functions
  # TODO: Add here new feature columns from the values that were created with our 'add_engineered_features' funcion.
]

## 6) Serving Input Receiver Function 

Same as before except the received tensors are wrapped with `add_engineered_features()`.

### Exercise 4

When building the `serving_input_receiver_fn()` we need to add our engineered features to the dictionary of `receiver_tensors` we will recieve at inference time. Complete the code to achieve this by using the `add_engineered_features` function from above.

In [ ]:
def serving_input_receiver_fn():
    receiver_tensors = {
        'dayofweek' : tf.placeholder(tf.int32, shape=[None]), # shape is vector to allow batch of requests
        'hourofday' : tf.placeholder(tf.int32, shape=[None]),
        'pickuplon' : tf.placeholder(tf.float32, shape=[None]), 
        'pickuplat' : tf.placeholder(tf.float32, shape=[None]),
        'dropofflat' : tf.placeholder(tf.float32, shape=[None]),
        'dropofflon' : tf.placeholder(tf.float32, shape=[None]),
    }
    
    features = # TODO: Add your code here. The result should be a dictionary.
    
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

## 7) Train and Evaluate

Same as before

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('taxi_trained')

In [ ]:
%%time
OUTDIR = 'taxi_trained/500'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.DNNRegressor(
    hidden_units = [10,10], # specify neural architecture
    feature_columns = feature_cols, 
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(
          tf_random_seed=1, # for reproducibility
          save_checkpoints_steps=100 # checkpoint every N steps
    ) 
)

# Add custom evaluation metric
def my_rmse(labels, predictions):
    pred_values = tf.squeeze(predictions['predictions'],axis=-1)
    return {'rmse': tf.metrics.root_mean_squared_error(labels, pred_values)}
model = tf.contrib.estimator.add_metrics(model, my_rmse) 
    
train_spec=tf.estimator.TrainSpec(
                   input_fn = lambda:train_input_fn('./taxi-train.csv'),
                   max_steps = 500)

exporter = tf.estimator.FinalExporter('exporter', serving_input_receiver_fn) # export SavedModel once at the end of training
# Note: alternatively use tf.estimator.BestExporter to export at every checkpoint that has lower loss than the previous checkpoint

eval_spec=tf.estimator.EvalSpec(
                   input_fn=lambda:eval_input_fn('./taxi-valid.csv'),
                   steps = None,
                   start_delay_secs=1, # wait at least N seconds before first evaluation (default 120)
                   throttle_secs=1, # wait at least N seconds before each subsequent evaluation (default 600)
                   exporters = exporter) # export SavedModel once at the end of training

tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## Results

Our RMSE is now **5.94**, our first significant improvement! If we look at the RMSE trend in TensorBoard it appears the model is still learning, so training past 500 steps would likely lower the RMSE even more. Let's run again, this time for 10x as many steps.

In [ ]:
%%time
OUTDIR = 'taxi_trained/5000'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.DNNRegressor(
    hidden_units = [10,10], # specify neural architecture
    feature_columns = feature_cols, 
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(
          tf_random_seed=1, # for reproducibility
          save_checkpoints_steps=500 # checkpoint every N steps
    ) 
)

# Add custom evaluation metric
def my_rmse(labels, predictions):
    pred_values = tf.squeeze(predictions['predictions'],axis=-1)
    return {'rmse': tf.metrics.root_mean_squared_error(labels, pred_values)}
model = tf.contrib.estimator.add_metrics(model, my_rmse) 
    
train_spec=tf.estimator.TrainSpec(
                   input_fn = lambda:train_input_fn('./taxi-train.csv'),
                   max_steps = 5000)

exporter = tf.estimator.FinalExporter('exporter', serving_input_receiver_fn) # export SavedModel once at the end of training

eval_spec=tf.estimator.EvalSpec(
                   input_fn=lambda:eval_input_fn('./taxi-valid.csv'),
                   steps = None,
                   start_delay_secs=1, # wait at least N seconds before first evaluation (default 120)
                   throttle_secs=1, # wait at least N seconds before each subsequent evaluation (default 600)
                   exporters = exporter) # export SavedModel once at the end of training

tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

## Results: 5000 steps

Our RMSE is now **4.13**! It looks like RMSE may still be reducing, but training is getting slow so we should move to the cloud if we want to train longer.

Also we haven't explored our hyperparameters much. Is our neural architecture of [10,10] optimal? 

In the next notebook we'll explore this.

## Cleanup

In [ ]:
if len(TensorBoard.list())>0:
  [TensorBoard().stop(pid)for pid in TensorBoard.list()['pid']]
else: print('No TensorBoard instances to stop')

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License